# Imports

In [3]:
import matplotlib.pyplot as plt
from skimage import data, filters
from skimage.color import rgb2gray
from skimage.io import imread
import os
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from skimage import measure
import torch
import torchmetrics
from torchmetrics.classification import JaccardIndex, Dice
import pandas as pd

# Binary Thresaholding

In [4]:
gtmasks_dirs = list(filter(lambda dname: dname.endswith('gtmasks'),os.listdir('Img')))
gtmasks_dirs = sorted(gtmasks_dirs, key=lambda dname: int(dname.split('_')[3]))
gtmasks_dict = {dname: os.listdir(f'Img/{dname}') for dname in gtmasks_dirs}
gtmasks_dict.keys()

dict_keys(['wire_images_video_1_gtmasks', 'wire_images_video_2_gtmasks', 'wire_images_video_3_gtmasks', 'wire_images_video_4_gtmasks', 'wire_images_video_5_gtmasks', 'wire_images_video_6_gtmasks', 'wire_images_video_7_gtmasks', 'wire_images_video_8_gtmasks'])

In [5]:
image_dirs = list(filter(lambda dname: dname[-1].isdigit(), os.listdir('Img')))
image_dirs = sorted(image_dirs, key=lambda dname: int(dname[-1]))
images_dict = {dname: os.listdir(f'Img/{dname}') for dname in image_dirs}
images_dict

{'wire_images_video_1': ['wire1_ultrasound_watertank.png',
  'wire2_ultrasound_watertank.png',
  'wire3_ultrasound_watertank.png',
  'wire4_ultrasound_watertank.png',
  'wire5_ultrasound_watertank.png',
  'wire6_ultrasound_watertank.png'],
 'wire_images_video_2': ['wire10_ultrasound_watertank.png',
  'wire11_ultrasound_watertank.png',
  'wire12_ultrasound_watertank.png',
  'wire13_ultrasound_watertank.png',
  'wire14_ultrasound_watertank.png',
  'wire15_ultrasound_watertank.png',
  'wire16_ultrasound_watertank.png',
  'wire17_ultrasound_watertank.png',
  'wire18_ultrasound_watertank.png',
  'wire19_ultrasound_watertank.png',
  'wire20_ultrasound_watertank.png',
  'wire21_ultrasound_watertank.png',
  'wire22_ultrasound_watertank.png',
  'wire23_ultrasound_watertank.png',
  'wire7_ultrasound_watertank.png'],
 'wire_images_video_3': ['wire24_ultrasound_watertank.png',
  'wire25_ultrasound_watertank.png',
  'wire26_ultrasound_watertank.png',
  'wire27_ultrasound_watertank.png',
  'wire28_u

## Thresholding Functions

In [109]:
def huang_threshold(image):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()  # Normalize the histogram
    
    # Cumulative sum and cumulative mean
    cum_sum = np.cumsum(hist_norm)
    cum_mean = np.cumsum(hist_norm * np.arange(256))
    
    # Overall mean
    mean_total = cum_mean[-1]
    
    entropy_values = []
    for t in range(1, 255):
        p1 = cum_sum[t]
        p2 = 1 - p1
        mean1 = cum_mean[t] / p1 if p1 != 0 else 0
        mean2 = (mean_total - cum_mean[t]) / p2 if p2 != 0 else 0
        
        # Entropy calculation
        entropy = -p1 * np.log(p1 + 1e-6) - p2 * np.log(p2 + 1e-6)
        entropy_values.append(entropy)
    
    # Find the threshold that minimizes the entropy
    threshold = np.argmin(entropy_values)
    return threshold


def max_entropy_threshold(image):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()  # Normalize the histogram
    
    # Cumulative sum and cumulative mean
    cum_sum = np.cumsum(hist_norm)
    cum_mean = np.cumsum(hist_norm * np.arange(256))
    
    # Overall mean
    mean_total = cum_mean[-1]
    
    entropy_values = []
    for t in range(1, 255):
        p1 = cum_sum[t]
        p2 = 1 - p1
        mean1 = cum_mean[t] / p1 if p1 != 0 else 0
        mean2 = (mean_total - cum_mean[t]) / p2 if p2 != 0 else 0
        
        # Entropy for foreground and background
        entropy1 = -p1 * np.log(p1 + 1e-6)
        entropy2 = -p2 * np.log(p2 + 1e-6)
        total_entropy = entropy1 + entropy2
        entropy_values.append(total_entropy)
    
    # Find the threshold that maximizes the entropy
    threshold = np.argmax(entropy_values)
    return threshold


def percentile_threshold(image, percentile):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()
    
    # Find the cumulative sum to get the percentile
    cum_sum = np.cumsum(hist_norm)
    threshold = np.searchsorted(cum_sum, percentile / 100.0)
    
    return threshold


def moments_threshold(image):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()  # Normalize the histogram
    
    total_mean = np.sum(np.arange(256) * hist_norm)
    best_threshold = 0
    min_moment_diff = float('inf')
    
    for t in range(1, 255):
        w0 = np.sum(hist_norm[:t])
        w1 = np.sum(hist_norm[t:])
        
        mean0 = np.sum(np.arange(t) * hist_norm[:t]) / w0 if w0 != 0 else 0
        mean1 = np.sum(np.arange(t, 256) * hist_norm[t:]) / w1 if w1 != 0 else 0
        
        moment_diff = (mean0 - total_mean) ** 2 + (mean1 - total_mean) ** 2
        if moment_diff < min_moment_diff:
            min_moment_diff = moment_diff
            best_threshold = t
    
    return best_threshold


def intermodes_threshold(image):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()  # Normalize the histogram
    
    # Iterate over all possible thresholds and calculate between-class variance
    max_variance = -1
    best_threshold = 0
    
    for t in range(1, 255):
        # Split the histogram into two parts: [0, t] and [t+1, 255]
        w0 = np.sum(hist_norm[:t])
        w1 = np.sum(hist_norm[t:])
        
        mean0 = np.sum(np.arange(t) * hist_norm[:t]) / w0 if w0 != 0 else 0
        mean1 = np.sum(np.arange(t, 256) * hist_norm[t:]) / w1 if w1 != 0 else 0
        
        # Calculate between-class variance
        variance = w0 * w1 * (mean0 - mean1) ** 2
        
        if variance > max_variance:
            max_variance = variance
            best_threshold = t
    
    return best_threshold


def renyi_entropy_threshold(image, alpha=2):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()  # Normalize the histogram
    
    # Calculate Renyi entropy
    entropy_values = []
    for t in range(1, 255):
        p1 = np.sum(hist_norm[:t])
        p2 = 1 - p1
        entropy = (1 / (1 - alpha)) * np.log(p1 ** alpha + p2 ** alpha + 1e-6)
        entropy_values.append(entropy)
    
    threshold = np.argmax(entropy_values)
    return threshold


def shanbhag_threshold(image):
    hist, bin_edges = np.histogram(image.flatten(), bins=256, range=(0, 255))
    hist_norm = hist / hist.sum()  # Normalize the histogram
    
    # Calculate cumulative sum and entropy
    cum_sum = np.cumsum(hist_norm)
    threshold_values = []
    
    for t in range(1, 255):
        p1 = cum_sum[t]
        p2 = 1 - p1
        entropy = -p1 * np.log(p1 + 1e-6) - p2 * np.log(p2 + 1e-6)
        threshold_values.append(entropy)
    
    threshold = np.argmax(threshold_values)
    return threshold


def apply_thresholding_method(image, method, threshold_value):
    """
    Apply the corresponding thresholding method to the image.
    This function supports various thresholding techniques based on the method name.
    """
    try:
        match method:
            case "Otsu":
                threshold = filters.threshold_otsu(image)
            case "Li":
                threshold = filters.threshold_li(image)
            case "Huang":
                threshold = huang_threshold(image)
            case "Intermodes":
                threshold = intermodes_threshold(image)
            case "IsoData":
                threshold = filters.threshold_isodata(image)
            case "MaxEntropy":
                threshold = max_entropy_threshold(image)
            case "Mean":
                threshold = filters.threshold_mean(image)
            case "Minimum":
                threshold = filters.threshold_minimum(image)
            case "Moments":
                threshold = moments_threshold(image)
            case "Percentile":
                threshold = percentile_threshold(image, percentile=threshold_value)
            case "RenyiEntropy":
                threshold = renyi_entropy_threshold(image)
            case "Shanbhag":
                threshold = shanbhag_threshold(image)
            case "Triangle":
                threshold = filters.threshold_triangle(image)
            case "Yen":
                threshold = filters.threshold_yen(image)
            case "Default":
                threshold = threshold_value
            case _:
                # If no specific method is matched, use the default threshold_value.
                raise ValueError(f"Unfimiliar method for binary thresholding: {method}")
        
    except AttributeError as e:
        raise ValueError(f"{method}")
    
    # Generate binary mask based on the threshold
    binary_mask = image > threshold
    return binary_mask

## General Functions

In [164]:
def get_image_path(folder_name, file_name):
    return os.path.join("Img", folder_name, file_name)

def display_image(image, title):
     # Display the filtered image
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

def filter_image_top(image):
   # Get the shape of the image
    height, width = image.shape    
    # Create a filter matrix with ones
    filter_matrix = np.ones((height, width))
    # Set rows y = 0 to y = 175 to zeros
    filter_matrix[0:176, :] = 0  # Note: Includes row 175
    # Multiply the filter matrix by the image
    filtered_image = image * filter_matrix
    return filtered_image

def is_8bit_image(image):
    """
    Check if the given image is an 8-bit grayscale or RGB image.

    Parameters:
        image (numpy.ndarray): The image to check.

    Returns:
        bool: True if the image is 8-bit, False otherwise.
    """
    if not isinstance(image, np.ndarray):
        raise ValueError("Input must be a numpy array.")
    
    # Check the data type and value range
    return image.dtype == np.uint8 and image.min() >= 0 and image.max() <= 255

def create_8_bit_image(image):
    return (image * 255).astype(np.uint8)

def print_scores(accuracies):
    def highlight_max(x):
        # Apply red color to the max values in each column
        return ['background-color: red' if v == x.max() else '' for v in x]

    results = []
    for method, evaluation_list in accuracies.items():
        num_of_images = len(evaluation_list)
        metrics = {f"avg_{metric}" : 0 for metric in evaluation_list[0]}
        # metrics = {
        # "avg_precision": 0,
        # "avg_recall": 0,
        # "avg_f1_score": 0,
        # "avg_weighted_accuracy": 0,
        # "avg_accuracy": 0,
        # "avg_dice_score": 0,
        # "avg_dice_score_class_1": 0,
        # "avg_jaccard_score": 0
        # }
        for evaluation_dict in evaluation_list:
            # Calculate the total scores for each meetric
            for metric, score in evaluation_dict.items():
                metrics[f"avg_{metric}"] += score

        # Calculate the average score for each metric    
        for metric, total_score in metrics.items():
            metrics[metric] = total_score / num_of_images
            
        # Print the results
        # res = f"Method: {method} | Stats over {num_of_images} images | "
        # for metric, avg_score in metrics.items():
        #     res += f"{metric}: {avg_score} | "
        # print(f"{res}\n")

        # Append the method and its metrics to the results list
        metrics['Method'] = method
        metrics['Num_of_images'] = num_of_images
        results.append(metrics)

    # Convert the results list into a pandas DataFrame
    df = pd.DataFrame(results)
    
    # Reorder columns for better readability
    df = df[['Method', 'Num_of_images'] + [col for col in df.columns if col not in ['Method', 'Num_of_images']]]

    # Apply the styling function to highlight the max values in each column
    styled_df = df.style.apply(highlight_max, subset=[col for col in df.columns if col not in ['Method', 'Num_of_images']], axis=0)

    # Display the styled table
    display(styled_df)

    # Print the table
    # print(df.to_string(index=False))



    # {'Default': [{'precision': 0.8170394864812294, 'recall': 0.9224755128036193, 'f1_score': 0.8665621389668262, 'weighted_accuracy': np.float64(0.04073768734087935), 'accuracy': np.float64(0.9938316345214844)}], 'Huang': [{'precision': 1.0, 'recall': 4.392322220758115e-05, 'f1_score': 8.784258608573436e-05, 'weighted_accuracy': np.float64(0.021241812020889483), 'accuracy': np.float64(0.9782886505126953)}], 'Intermodes': [{'precision': 0.9550029603315572, 'recall': 0.49593710194579876, 'f1_score': 0.6528476438276959, 'weighted_accuracy': np.float64(0.03176400321353867), 'accuracy': np.float64(0.9885482788085938)}], 'IsoData': [{'precision': 0.9527932048350213, 'recall': 0.5124083102736416, 'f1_score': 0.6664191254177259, 'weighted_accuracy': np.float64(0.03211291366005753), 'accuracy': np.float64(0.9888620376586914)}], 'Li': [{'precision': 0.8852159234772823, 'recall': 0.8292265120569245, 'f1_score': 0.8563069805415703, 'weighted_accuracy': np.float64(0.038803689674750785), 'accuracy': np.float64(0.99395751953125)}], 'MaxEntropy': [{'precision': 0.8170394864812294, 'recall': 0.9224755128036193, 'f1_score': 0.8665621389668262, 'weighted_accuracy': np.float64(0.04073768734087935), 'accuracy': np.float64(0.9938316345214844)}], 'Mean': [{'precision': 0.8478868607136985, 'recall': 0.9058725348091536, 'f1_score': 0.8759210889554268, 'weighted_accuracy': np.float64(0.040405794078651525), 'accuracy': np.float64(0.9944276809692383)}], 'Minimum': [{'precision': 1.0, 'recall': 0.11288268107348355, 'f1_score': 0.20286537474839167, 'weighted_accuracy': np.float64(0.023638606428903586), 'accuracy': np.float64(0.980738639831543)}], 'Otsu': [{'precision': 0.9550029603315572, 'recall': 0.49593710194579876, 'f1_score': 0.6528476438276959, 'weighted_accuracy': np.float64(0.03176400321353867), 'accuracy': np.float64(0.9885482788085938)}], 'Percentile': [{'precision': 0.8170394864812294, 'recall': 0.9224755128036193, 'f1_score': 0.8665621389668262, 'weighted_accuracy': np.float64(0.04073768734087935), 'accuracy': np.float64(0.9938316345214844)}], 'RenyiEntropy': [{'precision': 0.8170394864812294, 'recall': 0.9224755128036193, 'f1_score': 0.8665621389668262, 'weighted_accuracy': np.float64(0.04073768734087935), 'accuracy': np.float64(0.9938316345214844)}], 'Shanbhag': [{'precision': 0.8170394864812294, 'recall': 0.9224755128036193, 'f1_score': 0.8665621389668262, 'weighted_accuracy': np.float64(0.04073768734087935), 'accuracy': np.float64(0.9938316345214844)}], 'Triangle': [{'precision': 0.8478868607136985, 'recall': 0.9058725348091536, 'f1_score': 0.8759210889554268, 'weighted_accuracy': np.float64(0.040405794078651525), 'accuracy': np.float64(0.9944276809692383)}], 'Yen': [{'precision': 0.8673773251046567, 'recall': 0.8827689199279659, 'f1_score': 0.8750054421176369, 'weighted_accuracy': np.float64(0.03992803594337602), 'accuracy': np.float64(0.9945240020751953)}]}

    # MaxEntropy: {'precision': 0.8170394864812294, 'recall': 0.9224755128036193, 'f1_score': 0.8665621389668262, 'weighted accuracy': np.float64(0.04073768734087935), 'accuracy': np.float64(0.9938316345214844)}











    

## Evaluation Functions

In [168]:
def calculate_accuracy(ground_truth, prediction):
    """
    Calculate the accuracy of the predicted mask compared to the ground truth mask.

    Parameters:
    - ground_truth: 2D numpy array (8-bit) representing the ground truth mask.
    - prediction: 2D numpy array (8-bit) representing the predicted mask.

    Returns:
    - accuracy: A float value between 0 and 1 representing the accuracy.
    """
    # Ensure the inputs are numpy arrays
    ground_truth = np.array(ground_truth)
    prediction = np.array(prediction)

    # Ensure the masks are binary (0 and 1 only)
    # ground_truth = (ground_truth > 0).astype(int)
    # prediction = (prediction > 0).astype(int)
    
    # Check that both images have the same shape
    if ground_truth.shape != prediction.shape:
        raise ValueError("Ground truth and prediction must have the same shape")
    
    # Calculate the number of correct pixels
    correct_pixels = np.sum(ground_truth == prediction)
    
    # Calculate the total number of pixels
    total_pixels = ground_truth.size
    
    # Calculate the accuracy
    accuracy = correct_pixels / total_pixels
    return accuracy


def calculate_weighted_accuracy(gtm_image, binary_mask):
    """
    Calculates the weighted accuracy for binary segmentation.

    Parameters:
        gtm_image (np.ndarray): Ground truth binary mask (2D array).
        binary_mask (np.ndarray): Predicted binary mask (2D array).

    Returns:
        float: The weighted accuracy.
    """
    # Ensure the masks are binary (0 and 1 only)
    gtm_image = (gtm_image > 0).astype(int)
    binary_mask = (binary_mask > 0).astype(int)

    # Total number of pixels
    total_pixels = gtm_image.size

    # Count the number of white (1) and black (0) pixels in the ground truth
    num_white_pixels = gtm_image.sum()
    num_black_pixels = total_pixels - num_white_pixels

    # Calculate the percentage of black and white pixels
    P_b = num_black_pixels / total_pixels
    P_w = 1 - P_b  # P_w is complementary to P_b

    # Count true positives for white and black pixels
    T_w = ((gtm_image == 1) & (binary_mask == 1)).sum()  # True white pixels
    T_b = ((gtm_image == 0) & (binary_mask == 0)).sum()  # True black pixels

    # Weighted accuracy calculation
    weighted_acc = ((P_b * T_w) + (P_w * T_b)) / total_pixels

    return weighted_acc


# def calculate_dice_score(gtm_image, binary_mask):
#     """
#     Calculates the Dice Score for binary segmentation.

#     Parameters:
#         gtm_image (np.ndarray): Ground truth binary mask (2D array).
#         binary_mask (np.ndarray): Predicted binary mask (2D array).

#     Returns:
#         float: The Dice Score.
#     """
#     # Calculate the intersection and the sum of the ground truth and predicted masks
#     intersection = np.sum((gtm_image == binary_mask))
#     # union = np.sum(gtm_image) + np.sum(binary_mask)
#     union = gtm_image.size + binary_mask.size

#     # Calculate the Dice Score
#     if union == 0:
#         return 1.0  # If both are empty, consider it as a perfect match
#     else:
#         return 2 * intersection / union
    

def calculate_dice_score_for_1_class(gtm_image, binary_mask):
    """
    Calculates the Dice Score for the 1 class (white pixels) in binary segmentation.

    Parameters:
        gtm_image (np.ndarray): Ground truth binary mask (2D array).
        binary_mask (np.ndarray): Predicted binary mask (2D array).

    Returns:
        float: The Dice Score for the 1 class.
    """
    gtm_image = (gtm_image > 0).astype(int)
    binary_mask = (binary_mask > 0).astype(int)
    # Calculate the intersection where both masks have 1
    intersection = np.sum((gtm_image == 1) & (binary_mask == 1))

    # Calculate the total number of pixels where either mask has 1 (union)
    total_positive_pixels = np.sum((gtm_image == 1) | (binary_mask == 1))

    # Calculate the Dice Score for the 1 class (positive class)
    if total_positive_pixels == 0:
        return 1.0  # If there are no 1s in either mask, consider it as a perfect match

    else:
        return 2 * intersection / total_positive_pixels
    

def calculate_jaccard_index(gtm_image, binary_mask):
    """
    Calculates the Jaccard Index (IoU) for multiclass segmentation using torchmetrics.

    Parameters:
        gtm_image (np.ndarray): Ground truth multi-class mask (2D array for multiclass).
        binary_mask (np.ndarray): Predicted multi-class mask (2D array for multiclass).

    Returns:
        float: The Jaccard Index for the specified number of classes.
    """

    # Ensure that the values are binary (0 or 1), thresholding if necessary
    gtm_image = (gtm_image > 0).astype(np.int64)  # Convert to binary: 0 or 1
    binary_mask = (binary_mask > 0).astype(np.int64)  # Convert to binary: 0 or 1
    # Convert the numpy arrays to torch tensors
    gtm_image_tensor = torch.tensor(gtm_image, dtype=torch.int64)
    binary_mask_tensor = torch.tensor(binary_mask, dtype=torch.int64)

    # Initialize the JaccardIndex metric for multiclass with the specified number of classes
    jaccard = JaccardIndex(task="binary", average="weighted")

    # Compute and return the Jaccard Index for all classes
    return jaccard(binary_mask_tensor, gtm_image_tensor).cpu().numpy()


def calculate_dice_score(gtm_image, binary_mask):
    """
    Calculates the Dice Score for binary segmentation using torchmetrics.

    Parameters:
        gtm_image (np.ndarray): Ground truth binary mask (2D array).
        binary_mask (np.ndarray): Predicted binary mask (2D array).

    Returns:
        float: The Dice Score, which is a measure of similarity between the ground truth and predicted masks.
    """

    # Ensure that the values are binary (0 or 1), thresholding if necessary
    gtm_image = (gtm_image > 0).astype(np.int64)  # Convert to binary: 0 or 1
    binary_mask = (binary_mask > 0).astype(np.int64)  # Convert to binary: 0 or 1
    # Convert the numpy arrays to torch tensors
    gtm_image_tensor = torch.tensor(gtm_image, dtype=torch.int64)
    binary_mask_tensor = torch.tensor(binary_mask, dtype=torch.int64)

    # Initialize the Dice metric for binary class (2 classes: 0 and 1)
    try:
        dice = Dice(average="micro", num_classes=2)
    except ValueError as e:
        print(e)
    # Compute and return the Dice score for all classes
    return dice(binary_mask_tensor, gtm_image_tensor).cpu().numpy()

def evaluate_binary_segmentation(gtm_image, binary_mask):
    """
    Computes precision, recall, and F1 score for binary segmentation masks.

    Parameters:
        gtm_image (np.ndarray): Ground truth binary mask (2D array).
        binary_mask (np.ndarray): Predicted binary mask (2D array).

    Returns:
        dict: Dictionary containing precision, recall, and F1 score.
    """
    # Flatten the masks
    gtm_image_flat = gtm_image.flatten()
    binary_mask_flat = binary_mask.flatten()

    # Ensure binary values (0 and 1 only)
    gtm_image_flat = (gtm_image_flat > 0).astype(int)
    binary_mask_flat = (binary_mask_flat > 0).astype(int)

    # Calculate precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(gtm_image_flat, binary_mask_flat, average='binary')

    dice_score = calculate_dice_score(gtm_image, binary_mask)
    dice_score_class_1 = calculate_dice_score_for_1_class(gtm_image, binary_mask)
    jaccard_score = calculate_jaccard_index(gtm_image, binary_mask)
    accuracy = calculate_accuracy(gtm_image, binary_mask)
    weighted_accuracy = calculate_weighted_accuracy(gtm_image, binary_mask)

    return {"precision": precision, "recall": recall, "f1_score": f1, "dice_score": dice_score, "dice_score_class_1": dice_score_class_1, "jaccard_score": jaccard_score, "accuracy": accuracy, "weighted_accuracy": weighted_accuracy}


# Testing

In [12]:
# load dict for ground truth masks images
for dname, image_list in gtmasks_dict.items():
    for image in image_list:
        print(get_image_path(dname, image))
        break
    break

Img\wire_images_video_1_gtmasks\wire1_ultrasound_watertank_gtmask.png


In [21]:
# Thresholds from ImageJ
thresholds_imagej = {
    "Default": 45,
    "Huang": 6,
    "Intermodes": 113,
    "IsoData": 45,
    "Li": 10,
    "MaxEntropy": 140,
    "Mean": 5,
    "Minimum": 188,
    "Otsu": 45,
    "Percentile": 0,
    "RenyiEntropy": 6,
    "Shanbhag": 238,
    "Triangle": 2,
    "Yen": 3
}

In [169]:
accuracies = {method: [] for method in thresholds_imagej}
errors = []
for image_dir, gtmask_dir in zip(images_dict, gtmasks_dict):
    for image_name, gtmask_name in zip(images_dict[image_dir], gtmasks_dict[gtmask_dir]):
        im_path = get_image_path(image_dir, image_name)
        gtm_path = get_image_path(gtmask_dir, gtmask_name)
        image = rgb2gray(imread(im_path))
        # display_image(image, im_path)
        # Filters the top of the image and converts it to an 8-bit image
        filtered_image = create_8_bit_image(filter_image_top(image))
        # display_image(filtered_image, im_path)
        gtm_image = imread(gtm_path, as_gray=True)
        # display_image(gtm_image, gtm_path)
        # print(is_8bit_image(gtm_image))
        # print(is_8bit_image(filtered_image))
        # Generate ground truth masks using ImageJ thresholds
        for method, threshold in thresholds_imagej.items():
            threshold_normalized = threshold / 255.0  # Normalize threshold to match the image range
            # Creates a binary mask image of 8 bits
            # binary_mask = create_8_bit_image(filtered_image > threshold_normalized)
            try:
                binary_mask = create_8_bit_image(apply_thresholding_method(filtered_image, method, threshold_normalized))
                # display_image(binary_mask, f"Ground Truth Mask ({method}): Threshold = {threshold}")
                # print(is_8bit_image(binary_mask))
                # print(f"method: {method}     accuracy: {calculate_accuracy(gtm_image, binary_mask)}")
                
                evaluation_dict = evaluate_binary_segmentation(gtm_image, binary_mask)
                # display_image(gtm_image, gtm_path)
                # display_image(binary_mask, "binary mask")
                accuracies[method].append(evaluation_dict)
            except ValueError as e:
                errors.append(e)
                
        # print(accuracies)
        # print(errors)
print(accuracies)

print_scores(accuracies)

{'Default': [{'precision': 0.8170394864812294, 'recall': 0.9224755128036193, 'f1_score': 0.8665621389668262, 'dice_score': array(0.99383163, dtype=float32), 'dice_score_class_1': np.float64(1.5290862759373862), 'jaccard_score': array(0.7645431, dtype=float32), 'accuracy': np.float64(0.9938316345214844), 'weighted_accuracy': np.float64(0.04073768734087935)}, {'precision': 0.8165272995573045, 'recall': 0.9520028233633315, 'f1_score': 0.8790761146302218, 'dice_score': array(0.99433804, dtype=float32), 'dice_score_class_1': np.float64(1.5684849365846567), 'jaccard_score': array(0.78424245, dtype=float32), 'accuracy': np.float64(0.9943380355834961), 'weighted_accuracy': np.float64(0.04118597648994182)}, {'precision': 0.8346327451784982, 'recall': 0.8812824956672444, 'f1_score': 0.8573234984193888, 'dice_score': array(0.9948349, dtype=float32), 'dice_score_class_1': np.float64(1.50055330136481), 'jaccard_score': array(0.7502766, dtype=float32), 'accuracy': np.float64(0.9948348999023438), 'we

,Method,Num_of_images,avg_precision,avg_recall,avg_f1_score,avg_dice_score,avg_dice_score_class_1,avg_jaccard_score,avg_accuracy,avg_weighted_accuracy
0,Default,89,0.581364,0.980204,0.700773,0.989272,1.148346,0.574173,0.989273,0.029129
1,Huang,89,0.865169,0.001684,0.003346,0.984923,0.003369,0.001684,0.984923,0.014832
2,Intermodes,89,0.923954,0.530706,0.664707,0.992181,1.014858,0.507429,0.992182,0.022275
3,IsoData,89,0.923189,0.539174,0.670397,0.992251,1.028692,0.514346,0.992251,0.022354
4,Li,89,0.702423,0.938155,0.756027,0.992771,1.311902,0.655951,0.992771,0.028287
5,MaxEntropy,89,0.581364,0.980204,0.700773,0.989272,1.148346,0.574173,0.989273,0.029129
6,Mean,89,0.632617,0.974659,0.733179,0.991453,1.241203,0.620601,0.991453,0.029051
7,Minimum,89,0.923523,0.317875,0.446805,0.989745,0.623496,0.311748,0.989746,0.019670
8,Otsu,89,0.923543,0.535201,0.667694,0.992216,1.022134,0.511067,0.992216,0.022315
9,Percentile,89,0.581364,0.980204,0.700773,0.989272,1.148346,0.574173,0.989273,0.029129
